## Задание 2.2

### Формула PFound

$$pFound@K = \sum_{i=1}^{k} pLook[i]\ pRel[i]$$

$$pLook[1] = 1$$

$$pLook[i] = pLook[i-1]\ (1 - pRel[i-1])\ (1 - pBreak)$$

$$pBreak = 0.15$$

**Задача** - написать функцию, которая принимает на вход dataframe (после join), а на выходе дает средний pFound по всем query.
- Запрещается использовать циклы for для расчет метрики (как полностью, так и ее частей).
- Усложнение, если задача показалась легкой - попробуйте обойтись без groupby (не уверен, что это возможно, но вдруг вы справитесь)

##1.Базовое решение с циклами


Взято из решения задачи Yandex Cup.

In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Чтение данных
qid_query = pd.read_csv("/content/drive/MyDrive/Yandex Cup/hidden_task/qid_query.tsv", sep="\t", names=["qid", "query"])
qid_url_rating = pd.read_csv("/content/drive/MyDrive/Yandex Cup/hidden_task/qid_url_rating.tsv", sep="\t", names=["qid", "url", "rating"])
hostid_url = pd.read_csv("/content/drive/MyDrive/Yandex Cup/hidden_task/hostid_url.tsv", sep="\t", names=["hostid", "url"])

In [3]:
# Join таблиц qid_url_rating и hostid_url по url
qid_url_rating_hostid = pd.merge(qid_url_rating, hostid_url, on="url")

In [4]:
def plook(ind, rels):
    if ind == 0:
        return 1
    return plook(ind-1, rels)*(1-rels[ind-1])*(1-0.15)

In [5]:
def pfound(group):
    max_by_host = group.groupby("hostid")["rating"].max() # Максимальный рейтинг хоста
    top10 = max_by_host.sort_values(ascending=False)[:10] # Топ-10 url с наивысшим рейтингом
    pfound = 0
    for ind, val in enumerate(top10):
        pfound += val*plook(ind, top10.values)
    return pfound

In [6]:
qid_pfound = qid_url_rating_hostid.groupby('qid').apply(pfound) # Группировка по qid и вычисление pfound
qid_max = qid_pfound.idxmax() # qid с максимальным pfound

In [7]:
qid_query[qid_query["qid"] == qid_max]

,qid,query
14,692308,бесплатный просмотр камеди клаб


##2.Функция без циклов

In [8]:
def PFound(df: pd.DataFrame, k: int = 10, p_break: float = 0.15) -> float:
    # Группировка, чтобы оставить максимально релевантный документ относительно id хоста
    qid_hostid_max_rating = df.groupby(['qid', 'hostid'])['rating'].max().reset_index()

    # Сортировка по убыванию
    sorted_qid = qid_hostid_max_rating.sort_values(['qid', 'rating'], ascending=False)

    # Топ-k hostid для каждого qid по rating
    sorted_qid_top_k = sorted_qid.groupby(['qid']).head(k)

    # Добавление столбцов с промежуточными расчетами
    sorted_qid_top_k['1-pBreak'] = 1 - p_break
    sorted_qid_top_k['1-pRel'] = (1 - sorted_qid_top_k['rating']).shift(1)
    sorted_qid_top_k['sorted_num'] = sorted_qid_top_k.groupby('qid').cumcount()

    # Учитываем значение каждого первого эдемента
    sorted_qid_top_k['1-pBreak'] = np.where(sorted_qid_top_k['sorted_num'] == 0, 1, sorted_qid_top_k['1-pBreak'])
    sorted_qid_top_k['1-pRel'] = np.where(sorted_qid_top_k['sorted_num'] == 0, 1, sorted_qid_top_k['1-pRel'])

    # Столбцы для расчета pLook
    sorted_qid_top_k['(1-pRel)*(1-pBreak)'] = sorted_qid_top_k['1-pRel'] * sorted_qid_top_k['1-pBreak']
    sorted_qid_top_k['pLook'] = sorted_qid_top_k.groupby('qid')['(1-pRel)*(1-pBreak)'].cumprod()

    # Столбец с расчетом pFound
    sorted_qid_top_k['pFound'] = sorted_qid_top_k['pLook'] * sorted_qid_top_k['rating']

    # Средний pFound по всем query.
    pfound_sum = sorted_qid_top_k.groupby('qid')['pFound'].sum()
    return pfound_sum.mean()
    

##3.Проверка решения

In [9]:
# Рассчитанное функцией значение
avg_pfound = PFound(qid_url_rating_hostid)
print(avg_pfound)

0.5822199638393889


In [10]:
# Из решения Yandex Cup
qid_pfound.mean()

0.5822199638393889